<h1><center>Análise Dados</center></h1>
<h3><center>RPA para trazer as informações do banco do primeiro ano para o segundo</center></h3>

***

Banco de dados: 
* db_incluses (2 ano, postgresql e mongo)
* db_incluses (1 ano, postgresql)

Conteúdo deste notebook:
1. Imports
1. Trazendo valores da .env
1. Trazendo valores do banco do primeiro para o do segundo
    1. mongo
        1. trazendo as alterações da situacao_trabalhista
        1. trazendo as alterações da funcao
    1. postgresql
        1. trazendo as alterações do tipo_vaga
        1. trazendo as alterações do setor
        1. trazendo as alterações do tipo_arquivo
        1. trazendo as alterações da permissao_curso
        1. trazendo as alterações da permissao_vaga

## Imports

In [10]:
import os
import psycopg2
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv

## Trazendo valores da .env

In [11]:
# Esse comando carrega as variáveis de ambiente do arquivo .env para o ambiente Python, permitindo o uso seguro de credenciais e configurações sensíveis.
load_dotenv()

# Trando as valores para a conexão do bancoDB, partir do arquivo .env.
host_1ano = os.getenv("DB_HOST_1ano")
port_1ano = int(os.getenv("DB_PORT_1ano"))
user_1ano = os.getenv("DB_USER_1ano")
password_1ano = os.getenv("DB_PASSWORD_1ano")
database_1ano = os.getenv("DB_NAME_1ano")
db_url_1ano = os.getenv("DATABASE_URL_1ano")

host_2ano = os.getenv("DB_HOST_2ano")
port_2ano = int(os.getenv("DB_PORT_2ano"))
user_2ano = os.getenv("DB_USER_2ano")
password_2ano = os.getenv("DB_PASSWORD_2ano")
database_2ano = os.getenv("DB_NAME_2ano")
db_url_2ano = os.getenv("DATABASE_URL_2ano")

# Conecta ao MongoDB usando a URL obtida do arquivo .env.
conexao = MongoClient(os.getenv("DB_MONGO_URL"))

# Seleciona o banco de dados do MongoDB a partir do nome definido no arquivo .env.
db_name = conexao[os.getenv("DB_MONGO_NAME")]

# Seleciona a collection da situacao_trabalhista, do banco de dados MongoDB a partir do arquivo .env.
collection_situacao_trabalhista = db_name[os.getenv("DB_MONGO_COLLECTION_SITUACAO_TRABALHISTA")]

## Trazendo valores do banco do primeiro para o do segundo

### mongo

#### trazendo as alterações da situacao_trabalhista

In [12]:
try:
    # Conecta ao banco de dados do 1º ano.
    conn_1ano = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )

 # Trazendo todos os registros da tabela 'situacao_trabalhista' no banco do 1º ano e armazena em um DataFrame.
    df_situacao_trabalhista_1ano = pd.read_sql_query('SELECT * FROM situacao_trabalhista', conn_1ano)

    for index, row in df_situacao_trabalhista_1ano.iterrows():
        # Cria um filtro para buscar o documento correspondente ao 'id'.
        filtro = {"id": row['id']}  

        # Verifica se já existe um documento com o mesmo 'id'.
        if collection_funcoes.count_documents(filtro) > 0:
            # Se o documento existir, atualiza o campo 'nome' no MongoDB.
            collection_funcoes.update_one(filtro, {"$set": {"nome": row['nome']}})
        else:
            # Se o documento não existir, cria um novo documento com os campos 'id' e 'nome' e insere.
            situacao_trabalhista = {
                "id": row['id'],
                "nome": row['nome']
            }
            collection_situacao_trabalhista.insert_one(situacao_trabalhista)

    # Fecha a conexão com o banco de dados do 1º ano.
    conn_1ano.close()

except Exception as e:
    print("Erro ao conectar ao PostgreSQL:", e)


C:\Users\lucalucareli-ieg\AppData\Local\Temp\ipykernel_640\3977008798.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_situacao_trabalhista_1ano = pd.read_sql_query('SELECT * FROM situacao_trabalhista', conn_1ano)


## postgresql

#### trazendo as alterações do tipo_vaga

In [13]:
try:
    # Conecta ao banco de dados do 1º ano.
    conn_1ano = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )
    # Conecta ao banco de dados do 2º ano.
    conn_2ano = psycopg2.connect(
        user=user_2ano,
        host=host_2ano,
        port=port_2ano,
        password=password_2ano,
        database=database_2ano,
    )

    # Trazendo todos os registros da tabela 'tipo_vaga' no banco do 1º ano e armazena em um DataFrame.
    df_tipo_vaga_1ano = pd.read_sql_query('''SELECT * FROM tipo_vaga''', conn_1ano)

    # Cria um cursor para executar comandos no banco de dados do 2º ano.
    cursor_2ano = conn_2ano.cursor()

    for i in range(len(df_tipo_vaga_1ano)):
        # Define os valores para a atualização da tabela 'tipo_vaga' no banco do 2º ano (nome e id).
        values = (df_tipo_vaga_1ano['nome'][i], df_tipo_vaga_1ano['id'][i])

        # Prepara e executa a query de atualização no banco de dados do 2º ano, atualizando o nome baseado no id.
        update_query = """UPDATE public.tipo_vaga
                          SET nome = %s
                          WHERE id = %s;"""
        cursor_2ano.execute(update_query, values)

        # Se não tiver atualização, insere um novo registro com id e permissão.
        if cursor_2ano.rowcount == 0:  
            insert_query = """INSERT INTO public.tipo_vaga (id, nome)
                              VALUES (%s, %s);"""
            cursor_2ano.execute(insert_query, (df_tipo_vaga_1ano['id'][i], df_tipo_vaga_1ano['nome'][i]))

    # Confirma as alterações no banco de dados do 2º ano.
    conn_2ano.commit()

    # Fecha as conexões com os dois bancos de dados.
    conn_2ano.close()
    conn_1ano.close()

except Exception as e:
    print("Error connecting to PostgreSQL:", e)


C:\Users\lucalucareli-ieg\AppData\Local\Temp\ipykernel_640\1885338333.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tipo_vaga_1ano = pd.read_sql_query('''SELECT * FROM tipo_vaga''', conn_1ano)


#### trazendo as alterações do setor

In [14]:
try:
    # Conecta ao banco de dados do 1º ano.
    conn_1ano = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )
    # Conecta ao banco de dados do 2º ano.
    conn_2ano = psycopg2.connect(
        user=user_2ano,
        host=host_2ano,
        port=port_2ano,
        password=password_2ano,
        database=database_2ano,
    )

    # Trazendo todos os registros da tabela 'setor' no banco do 1º ano e armazena em um DataFrame.
    df_setor_1ano = pd.read_sql_query('''SELECT * FROM setor''', conn_1ano)

    # Cria um cursor para executar comandos no banco de dados do 2º ano.
    cursor_2ano = conn_2ano.cursor()

    for i in range(len(df_setor_1ano)):
        # Define os valores para a atualização da tabela 'setor' no banco do 2º ano (nome e id).
        values = (df_setor_1ano['nome'][i], df_setor_1ano['id'][i])

        # Prepara e executa a query de atualização no banco de dados do 2º ano, atualizando o nome baseado no id.
        update_query = """UPDATE public.setor
                          SET nome = %s
                          WHERE id = %s;"""
        cursor_2ano.execute(update_query, values)

        # Se não tiver atualização, insere um novo registro com id e permissão.
        if cursor_2ano.rowcount == 0:  
            insert_query = """INSERT INTO public.setor (id, nome)
                              VALUES (%s, %s);"""
            cursor_2ano.execute(insert_query, (df_setor_1ano['id'][i], df_setor_1ano['nome'][i]))

    # Confirma as alterações no banco de dados do 2º ano.
    conn_2ano.commit()

    # Fecha as conexões com os dois bancos de dados.
    conn_2ano.close()
    conn_1ano.close()

except Exception as e:
    print("Error connecting to PostgreSQL:", e)

C:\Users\lucalucareli-ieg\AppData\Local\Temp\ipykernel_640\1729403630.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_setor_1ano = pd.read_sql_query('''SELECT * FROM setor''', conn_1ano)


#### trazendo as alterações do tipo_arquivo

In [15]:
try:
    # Conecta ao banco de dados do 1º ano.
    conn_1ano = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )
    # Conecta ao banco de dados do 2º ano.
    conn_2ano = psycopg2.connect(
        user=user_2ano,
        host=host_2ano,
        port=port_2ano,
        password=password_2ano,
        database=database_2ano,
    )

    # Trazendo todos os registros da tabela 'tipo_arquivo' no banco do 1º ano e armazena em um DataFrame.
    df_tipo_arquivo_1ano = pd.read_sql_query('''SELECT * FROM tipo_arquivo''', conn_1ano)

    # Cria um cursor para executar comandos no banco de dados do 2º ano.
    cursor_2ano = conn_2ano.cursor()

    for i in range(len(df_tipo_arquivo_1ano)):
        # Define os valores para a atualização da tabela 'tipo_arquivo' no banco do 2º ano (nome e id).
        values = (df_tipo_arquivo_1ano['nome'][i], df_tipo_arquivo_1ano['id'][i])

        # Prepara e executa a query de atualização no banco de dados do 2º ano, atualizando o nome baseado no id.
        update_query = """UPDATE public.tipo_arquivo
                          SET nome = %s
                          WHERE id = %s;"""
        cursor_2ano.execute(update_query, values)

        # Se não tiver atualização, insere um novo registro com id e permissão.
        if cursor_2ano.rowcount == 0:  
            insert_query = """INSERT INTO public.tipo_arquivo (id, nome)
                              VALUES (%s, %s);"""
            cursor_2ano.execute(insert_query, (df_tipo_arquivo_1ano['id'][i], df_tipo_arquivo_1ano['nome'][i]))

    # Confirma as alterações no banco de dados do 2º ano.
    conn_2ano.commit()

    # Fecha as conexões com os dois bancos de dados.
    conn_2ano.close()
    conn_1ano.close()

except Exception as e:
    print("Error connecting to PostgreSQL:", e)


C:\Users\lucalucareli-ieg\AppData\Local\Temp\ipykernel_640\1230204521.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tipo_arquivo_1ano = pd.read_sql_query('''SELECT * FROM tipo_arquivo''', conn_1ano)


#### trazendo as alterações da permissao_curso

In [16]:
try:
    # Conecta ao banco de dados do 1º ano.
    conn_1ano = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )
    # Conecta ao banco de dados do 2º ano.
    conn_2ano = psycopg2.connect(
        user=user_2ano,
        host=host_2ano,
        port=port_2ano,
        password=password_2ano,
        database=database_2ano,
    )

    # Trazendo todos os registros da tabela 'permissao_curso' no banco do 1º ano e armazena em um DataFrame.
    df_permissao_curso_1ano = pd.read_sql_query('''SELECT * FROM permissao_curso''', conn_1ano)

    # Cria um cursor para executar comandos no banco de dados do 2º ano.
    cursor_2ano = conn_2ano.cursor()

    for i in range(len(df_permissao_curso_1ano)):
        # Define os valores para a atualização da tabela 'permissao_curso' no banco do 2º ano (nome e id).
        values = (df_permissao_curso_1ano['nome'][i], df_permissao_curso_1ano['id'][i])

        # Prepara e executa a query de atualização no banco de dados do 2º ano, atualizando o nome baseado no id.
        update_query = """UPDATE public.permissao_curso
                          SET permissao = %s
                          WHERE id = %s;"""
        cursor_2ano.execute(update_query, values)

        # Se não tiver atualização, insere um novo registro com id e permissão.
        if cursor_2ano.rowcount == 0:  
            insert_query = """INSERT INTO public.permissao_curso (id, permissao)
                              VALUES (%s, %s);"""
            cursor_2ano.execute(insert_query, (df_permissao_curso_1ano['id'][i], df_permissao_curso_1ano['permissao'][i]))

    # Confirma as alterações no banco de dados do 2º ano.
    conn_2ano.commit()

    # Fecha as conexões com os dois bancos de dados.
    conn_2ano.close()
    conn_1ano.close()

except Exception as e:
    print("Error connecting to PostgreSQL:", e)


C:\Users\lucalucareli-ieg\AppData\Local\Temp\ipykernel_640\3228341067.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_permissao_curso_1ano = pd.read_sql_query('''SELECT * FROM permissao_curso''', conn_1ano)


#### trazendo as alterações da permissao_vaga

In [17]:
try:
    # Conecta ao banco de dados do 1º ano.
    conn_1ano = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )
    # Conecta ao banco de dados do 2º ano.
    conn_2ano = psycopg2.connect(
        user=user_2ano,
        host=host_2ano,
        port=port_2ano,
        password=password_2ano,
        database=database_2ano,
    )

    # Trazendo todos os registros da tabela 'permissao_vaga' no banco do 1º ano e armazena em um DataFrame.
    df_permissao_vaga_1ano = pd.read_sql_query('''SELECT * FROM permissao_vaga''', conn_1ano)

    # Cria um cursor para executar comandos no banco de dados do 2º ano.
    cursor_2ano = conn_2ano.cursor()

    for i in range(len(df_permissao_vaga_1ano)):
        # Define os valores para a atualização da tabela 'permissao_vaga' no banco do 2º ano (nome e id).
        values = (df_permissao_vaga_1ano['nome'][i], df_permissao_vaga_1ano['id'][i])

        # Prepara e executa a query de atualização no banco de dados do 2º ano, atualizando o nome baseado no id.
        update_query = """UPDATE public.permissao_vaga
                          SET permissao = %s
                          WHERE id = %s;"""
        cursor_2ano.execute(update_query, values)

        # Se não tiver atualização, insere um novo registro com id e permissão.
        if cursor_2ano.rowcount == 0:  
            insert_query = """INSERT INTO public.permissao_vaga (id, permissao)
                              VALUES (%s, %s);"""
            cursor_2ano.execute(insert_query, (df_permissao_vaga_1ano['id'][i], df_permissao_vaga_1ano['permissao'][i]))

    # Confirma as alterações no banco de dados do 2º ano.
    conn_2ano.commit()

    # Fecha as conexões com os dois bancos de dados.
    conn_2ano.close()
    conn_1ano.close()

except Exception as e:
    print("Error connecting to PostgreSQL:", e)


C:\Users\lucalucareli-ieg\AppData\Local\Temp\ipykernel_640\524051283.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_permissao_vaga_1ano = pd.read_sql_query('''SELECT * FROM permissao_vaga''', conn_1ano)


## funcao

In [10]:
try:
    conn_1ano = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )

    df_funcao_1ano = pd.read_sql_query('SELECT * FROM funcao', conn_1ano)

    display(df_funcao_1ano)

    for index, row in df_funcao_1ano.iterrows():
        filtro = {"id": row['id']}  

        if collection_funcoes.count_documents(filtro) > 0:

            collection_funcoes.update_one(filtro, {"$set": {"nome": row['nome']}})
        else:
            funcao = {
                "id": row['id'],
                "nome": row['nome']
            }
            collection_funcoes.insert_one(funcao)

    conn_1ano.close()

except Exception as e:
    print("Erro ao conectar ao PostgreSQL:", e)

Erro ao conectar ao PostgreSQL: Execution failed on sql 'SELECT * FROM funcao': relation "funcao" does not exist
LINE 1: SELECT * FROM funcao
                      ^



C:\Users\lucalucareli-ieg\AppData\Local\Temp\ipykernel_23424\3251264320.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_funcao_1ano = pd.read_sql_query('SELECT * FROM funcao', conn_1ano)
